# Homework 2 Part 2 - Model Based Reinforcement Learning

***

Written by Albert Wilcox

In this homework, you'll implement [PETS](https://arxiv.org/abs/1805.12114), a popular model for simple MBRL tasks.


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import einops

import gym
import numpy as np
from loguru import logger
import matplotlib.pyplot as plt
from IPython.display import Image
from tqdm import tqdm, trange

from typing import Tuple, Optional

from src.utils import (
    get_device,
    set_seed,
    demo_policy,
    save_frames_as_gif
)
# Do not remove the following import
import src.cartpole_env
from src.mpc import MPC
from src.mbrl_utils import sample_rollout
from src.cartpole_env import CartpoleConfigModule
from src.mbrl_sampler import MBRLSampler

plt.ion()

In [ ]:
SEED: int = 42
ENVIRONMENT_NAME: str='MBRLCartpole-v0'

# torch related defaults
DEVICE = get_device()
torch.set_default_dtype(torch.float32)

# Use random seeds for reproducibility
set_seed(SEED)

As before, we start by initializing the environment and printing some useful information.

In [ ]:
env = gym.make(ENVIRONMENT_NAME)

# get the state and action dimensions
action_dimension = env.action_space.shape[0]
state_dimension = env.observation_space.shape[0]

logger.info(f'Action Dimension: {action_dimension}')
logger.info(f'Action High: {env.action_space.high}')
logger.info(f'Action Low: {env.action_space.low}')
logger.info(f'State Dimension: {state_dimension}')

### Part 1 - PETS

In this part you'll implement the PETS (Chua et al.) dynamics model and use it for model-based control (MPC). There are several important components of this pipeline:
 * The dynamics model, discussed in more detail in Chua et al, learns to predict the next state $s_{t+1}$ conditioned on the current state-action pair $(s_t, a_t)$.
 * The cost function outputs the cost of a planned state. In the case of this environment, we provide a ground truth cost function (negative velocity), but in more complicated environments where no ground truth cost function is available it is common to learn it.
 * Cross entropy method (CEM) is a gradient-free evolutionary optimizer. We use it to optimize sequences of actions, and evaluate these sequences of actions by predicting future states after rolling out the planned actions and computing the total cost of the predicted rollout under the cost function.

The first step is to set up our probabilistic dynamics model. As described in Chua et al, this should take in a state and action and output `mean` and `log_std` for a Gaussian distribution over possible future states.

In [ ]:
from src.networks import network

class DynamicsModel(nn.Module):
    def __init__(self, 
                 state_dimension: int, 
                 action_dimension: int,
                 min_log_std: float = -5,
                 max_log_std: float = 1,
                 ):
        super(DynamicsModel, self).__init__()
        
        # TODO: fill in the parameters to initialize the prediction network
        
    
    def forward(self, state: torch.Tensor, action: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Forward pass of the dynamics network. Should return mean and log_std of the next state distribution

        Args:
            state (torch.Tensor): The input state.
            action (torch.Tensor): The input action.

        Returns:
            Tuple[torch.Tensor, torch.Tensor]: The tuple (mean, log_std) of the distribution
        """

        # TODO: predict the mean and log_std of the next state distribution as described above

        return mean, log_std
        

The next step is to create an ensemble of dynamics models. There are better ways to implement this, but for the purposes of this assignment we'll simply maintain a list of models and loop through them at inference time. If you have access to a GPU and are interested in speeding up your implementation, you might want to check out https://pytorch.org/tutorials/intermediate/ensembling.html.

There are several ways to handle data for ensembles, such as partitioning the dataset or training each network on different minibatches from the same dataset. For the purposes of this homework we'll use the easiest option, which is to train each model on the same minibatches. Due to the difference in randomly initialized NN weights, this still provides enough variance across models to give us some benefits.

TODOs for this section:
 * Fill in the `forward` function of the dynamics model to predict the mean and log_std from each member of the ensemble for a single batch of states and actions. This is for use during training
 * Fill in the `compute_cost` function which takes in a single state and a batch of action trajectory candidates and computes the expected cost for each one by rolling out the dynamics model. You should do this using the TS-1 algorithm from Chua et al, meaning for each step you randomly sample a dynamics model from the ensemble. Note the parameter `n_particles`. For each action trajectory candidate, you should sample `n_particles` trajectories and compute the mean between their costs.
 * Note: the `compute_cost` function will involve creating some large tensors. For tensors with many dimensions to keep track of, I would highly recommend using the `einops` library for rearranging / tiling / etc.

In [ ]:
class EnsembleDynamicsModel(nn.Module):
    def __init__(self, state_dimension: int, action_dimension: int, n_ensemble: int):
        super(EnsembleDynamicsModel, self).__init__()
        self.num_nets = n_ensemble

        # TODO: initialize an ensemble of dynamics models
        # Hint: You should store the models in an nn.ModuleList so that they appear when we do dynamics_model.parameters()
    
    def forward(self, state: torch.Tensor, action: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Forward pass of the dynamics network. Should return mean and log_std of the next state distribution for each model in the ensemble

        Args:
            state (torch.Tensor): The input state, shape (B, n_ensemble, S)
            action (torch.Tensor): The input action, shape (B, n_ensemble, A)

        Returns:
            Tuple[torch.Tensor, torch.Tensor]: The tuple (mean, log_std) of the distributions where each have shape (B, n_ensemble, S)
        """

        # TODO: predict the next state as described above

        return mean, log_std
        
    def compute_cost(
            self, 
            state: torch.Tensor, 
            actions: torch.Tensor,
            obs_cost_fn,
            act_cost_fn,
            n_particles: int = 20,
        ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Given a state and a 

        Args:
            state (torch.Tensor): The input state, shape (S,)
            actions (torch.Tensor): The action sequence candidates, shape (N, H, A)
            obs_cost_fn: A function which takes in a batch of states and returns the cost of each one
            act_cost_fn: A function which takes in a batch of actions and returns the cost of each one
            n_particles (int): how many particles to sample for each action sequence

        Returns:
            torch.Tensor: Expected cost for each action candidate, shape (N,)
        """
        n_candidates, horizon, _ = actions.shape

        # TODO: predict the trajectory using the TS-1 algorithm from Chua et al
        # Hint: You may have issues with NaN values. To deal with this, use the reparameterization trick
        #       to sample and then replace NaN costs with a high number

        
        return costs


Now that we've set up everything, the last step is to train our model. In the following block we provide some hyperparameters, the ground truth cost functions, and the skeleton of the training loop. You'll need to implement the loss function yourself.

Unfortunately MPC is quite slow to run, especially on a CPU. Thus, we've provided you an offline dataset so that you don't need to run the MPC policy to collect online data. The dataset should be sufficient to achieve a reward greater than 150 with a correct implementation.

Note: Our implementation achieved a validation loss <0.1.

The hyperparameters we provide should work well enough, but if you have access to a GPU you can improve performance by increasing `n_particles`, `popsize` and `num_elites`.

In [ ]:
################################## Hyper-parameters #########################################

EPOCHS = 150
EVAL_FREQ = 30
TASK_HORIZON = 200

plan_hor = 25
n_particles = 10
batch_size = 32
n_ensemble = 5
maxiters = 5
popsize = 100
num_elites = 10

################################### Cost Functions ###########################################

sampler = MBRLSampler(torch.load('data.pkl'), n_ensemble, batch_size, DEVICE)

# To make things faster for you we're providing an offline dataset that should be sufficient
rollouts = torch.load('data.pkl')
all_obs = np.concatenate([rollout['obs'] for rollout in rollouts], axis=0)
all_act = np.concatenate([rollout['act'] for rollout in rollouts], axis=0)
all_next_obs = np.concatenate([rollout['next_obs'] for rollout in rollouts], axis=0)

config = CartpoleConfigModule(DEVICE)
dynamics_model = EnsembleDynamicsModel(state_dimension, action_dimension, n_ensemble).to(DEVICE)
optimizer = Adam(dynamics_model.parameters(), 1e-3, weight_decay=1e-4)
policy = MPC(
    observation_space=env.observation_space,
    action_space=env.action_space,
    obs_cost_fn=config.obs_cost_fn,
    act_cost_fn=config.ac_cost_fn,
    dynamics_model=dynamics_model,
    plan_hor=plan_hor,
    n_particles=n_particles,
    max_iters=maxiters,
    popsize=popsize,
    num_elites=num_elites,
    alpha=0.1,
    device=DEVICE
)

data_len = all_obs.shape[0]

epoch_range = trange(EPOCHS, unit="epoch(s)", desc="Network training")
num_batch = int(np.ceil(data_len / batch_size))
result = None
rews = []

for epoch in epoch_range:

    for obs, act, next_obs in sampler:

        # TODO: compute NLL loss and update the dynamics model
        pass

    # Compute validation MSE loss
    # Note: this is a different loss function than the one you should implement to update the model
    val_obs, val_act, val_next_obs = sampler.get_val_data()
    mean, _ = dynamics_model(val_obs, val_act)
    mse_losses = ((mean - val_next_obs) ** 2).mean()

    epoch_range.set_postfix({
        "Training loss": mse_losses.item(),
        'Reward': result
    })

    # Sample an eval rollout. Note: If you are not using a GPU you should comment this out and only run eval once
    if (epoch + 1) % EVAL_FREQ == 0:
        info = sample_rollout(
            env,
            TASK_HORIZON,
            policy=policy,
        )
        result = info['reward_sum']
        rews.append(result)

torch.save(dynamics_model.state_dict(), 'pets_checkpoint.pth')


Now that you've finished training your dynamics model we can visualize our MPC policy and print out the final reward.

Note: If you're running this on a CPU it will likely be quite slow. I would suggest visualizing a much shorter episode and making sure that the policy looks right before running the full 200 step eval.

In [ ]:
dynamics_model.load_state_dict(torch.load('pets_checkpoint.pth'))
dynamics_model = dynamics_model.to(DEVICE)
frames, total_reward = demo_policy(policy, environment_name=ENVIRONMENT_NAME, steps=200)
gif_path = save_frames_as_gif(frames, method_name='pets')
print('Total Reward:', total_reward)
Image(open(gif_path,'rb').read())

Congrats on finishing the MBRL portion of Assignment 2! Hopefully you enjoyed yourself. Make sure that the visualizations are showing, an eval with a success rate greater than 150 is showing above, and that the `pets_policy.gif` is present in the outputs folder.

When you're done: export this notebook as an **HTML file** for final submission.